# Maryland bridges analysis

By [Christine Zhang](mailto:czhang@baltsun.com)

The Baltimore Sun conducted an analysis of bridge conditions in Maryland, with a focus on bridges in the Baltimore area. The Sun looked at which bridges were listed in "poor" condition by the Federal Highway Administration (FHA) — and in particular, which have been "poor" for several years.

The results were reported in a October 8, 2018 Baltimore Sun story titled ["How safe are Maryland's bridges?"](http://www.baltimoresun.com/news/maryland/bs-md-bridge-collapse-maryland-20180815-story.html)

Here are the key findings of the data analysis, which is documented below and based on data from the FHA’s 2017 and 2007 [National Bridge Inventory](https://www.fhwa.dot.gov/bridge/nbi.cfm):

- The median age of Maryland's bridges is 46 years.
- 288 (5.4 percent) of the bridges in Maryland were classified as being in both "poor" and "structurally deficient" condition.
- 66 (2.6 percent) of the bridges owned or maintained by the State Highway Administration were listed as being in "poor" condition.
- Baltimore city owns all of the untolled bridges within its boundaries.
- 33 (13 percent) of city-owned bridges in Baltimore were listed in "poor" condition.
- 20 city-owned bridges in Baltimore have been in "poor" condition for 10 years or more.
- There were 36 bridges listed in “poor” condition in Baltimore County.
- 11 bridges in Baltimore County have been in “poor” condition for 10 years or more.
- 21 bridges in Harford County were listed in “poor” condition.
- 11 bridges in Harford County have been in “poor” condition for 10 years or more.
- The State Highway Administration maintains roughly half of Maryland’s more than 5,000 bridges.
- The other half of the state’s bridges are owned by up to 15 different types of agencies, including county highway agencies, city or municipal highway agencies, state toll authorities, private owners and others.
- There is a bridge owned by NASA — a federal access road in Prince George’s County.

## How we did it

### Run `01_processing.ipynb`

The raw data files from FHA were pre-processed prior to the analysis. See the processing notebook [`01_processing.ipynb`](https://github.com/baltimore-sun-data/bridge-data/blob/master/01_processing.ipynb) for more details.

### Import R data analysis libraries

In [1]:
suppressMessages(library('tidyverse'))
suppressMessages(library('feather'))

Read in the 2017 data for analysis.

In [2]:
data.2017.clean <- read_feather('output/data_2017_clean.feather')

In [3]:
names(data.2017.clean)

[1] "STRUCTURE_NUMBER_008"         "county"                      
 [3] "combinedfips"                 "location"                    
 [5] "intersects"                   "carries"                     
 [7] "route_type"                   "service_level"               
 [9] "responsibility"               "owner"                       
[11] "op_status"                    "struct_deficient"            
[13] "struct_deficient_old"         "bridge_condition"            
[15] "SUFFICIENCY_RATING"           "lowest_condition_rating"     
[17] "total_poor_conditions"        "deck_condition"              
[19] "superstruct_condition"        "substruct_condition"         
[21] "culvert_condition"            "deck_condition_poor"         
[23] "superstruct_condition_poor"   "substruct_condition_poor"    
[25] "culvert_condition_poor"       "avg_daily_traffic"           
[27] "avg_daily_traffic_yr"         "avg_daily_truck_traffic_perc"
[29] "work_proposed"                "work_responsibility"         
[31] "inspection_mo"                "inspection_yr"               
[33] "yr_reconstructed"             "yr_built"                    
[35] "inspection_freq"              "bridge_improve_cost"         
[37] "roadway_improve_cost"         "total_improve_cost"          
[39] "yr_cost_estimate"             "avg_daily_traffic_future"    
[41] "avg_daily_traffic_future_yr"  "scour_critical"              
[43] "national_register"            "LAT_016"                     
[45] "LONG_017"                     "SCOUR_CRITICAL_113"          
[47] "COUNTY_CODE_003"              "structural_eval"             
[49] "waterway_eval"

For more information about the meanings of these column names, see the processing notebook [`01_processing.ipynb`](https://github.com/baltimore-sun-data/bridge-data/blob/master/01_processing.ipynb).

### Finding: The median age of Maryland's bridges is 46 years.

Calculate the age of each bridge by subtracting the current year, 2018, from the year the bridge was built.

In [4]:
data.2017.clean$age <- 2018 - data.2017.clean$yr_built

In [5]:
summary(data.2017.clean$age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00   28.00   46.00   48.27   61.00  209.00 

In [6]:
print(paste("The median age of Maryland's bridges is", 
            summary(data.2017.clean$age)['Median'],
            "years."))

[1] "The median age of Maryland's bridges is 46 years."


### Finding: 288 (5.4 percent) of the bridges in Maryland were classified as being in both "poor" and "structurally deficient" condition.

Group the data by `bridge_condition` and calculate the total number and percentage of bridges in each category.

In [7]:
md.conditions <- data.2017.clean %>% group_by(bridge_condition) %>% 
                                     summarise(n = n()) %>% 
                                     mutate(perc = n/sum(n) * 100)

md.conditions

bridge_condition,n,perc
Fair,3275,61.387067
Good,1772,33.214620
Poor,288,5.398313


In [8]:
print(paste("There were",
            md.conditions[md.conditions$bridge_condition == 'Poor',]$n, 
            "bridges listed in 'poor' condition in Maryland. That's", 
            round(md.conditions[md.conditions$bridge_condition == 'Poor',]$perc, 1), 
            "percent of all bridges in Maryland."))

[1] "There were 288 bridges listed in 'poor' condition in Maryland. That's 5.4 percent of all bridges in Maryland."


Per FHA, the formula for the "structurally deficient" classification is equivalent to the formula for the "poor" classification https://www.fhwa.dot.gov/bridge/britab.cfm. We can check this, just in case:

In [9]:
md.struct_def <- data.2017.clean %>% group_by(struct_deficient) %>% 
                                     summarise(n = n()) %>% 
                                     mutate(perc = n/sum(n) * 100)

In [10]:
print(paste("There were",
            md.struct_def[md.struct_def$struct_deficient == 1,]$n, 
            "bridges listed in structurally deficient condition in Maryland under the new definition of 'structurally deficient.' That's", 
            round(md.struct_def[md.struct_def$struct_deficient == 1,]$perc, 1), 
            "percent of all bridges in Maryland."))

[1] "There were 288 bridges listed in structurally deficient condition in Maryland under the new definition of 'structurally deficient.' That's 5.4 percent of all bridges in Maryland."


Note that using the previous definition of "structurally deficient" would result in 300 bridges being classified as "structurally deficient." This is because there are two additional items that went into the previous "structurally deficient" rating: Item 67 (Structural Condition) and Item 71 (Waterway Adequacy). The FHA's "Deficient Bridges by Highway System 2017" page uses the previous definition and comes up with 300 "deficient" bridges for Maryland: https://www.fhwa.dot.gov/bridge/nbi/no10/defbr17.cfm (archived link [here](https://web.archive.org/web/20180701162630/https://www.fhwa.dot.gov/bridge/nbi/no10/defbr17.cfm)).

In [11]:
md.struct_def_old <- data.2017.clean %>% group_by(struct_deficient_old) %>% 
                                     summarise(n = n()) %>% 
                                     mutate(perc = n/sum(n) * 100)

In [12]:
print(paste("There were",
            md.struct_def_old[md.struct_def_old$struct_deficient_old == 1,]$n, 
            "bridges in structurally deficient condition in Maryland under the old definition of 'structurally deficient.' That's", 
            round(md.struct_def_old[md.struct_def_old$struct_deficient_old == 1,]$perc, 1), 
            "percent of all bridges in Maryland."))

[1] "There were 300 bridges in structurally deficient condition in Maryland under the old definition of 'structurally deficient.' That's 5.6 percent of all bridges in Maryland."


In the story, we use the updated definition — 288 (5.4 percent) of bridges.

For more details on how the classification is calculated, see the processing notebook [`01_processing.ipynb`](https://github.com/baltimore-sun-data/bridge-data/blob/master/01_processing.ipynb).

### Finding: Nationally, 7.7 percent of bridges were classified as being in "Poor" condition.

This number is calculated from the totals columns on the "Bridge Condition by Highway System 2017" page of the FHA website: https://www.fhwa.dot.gov/bridge/nbi/no10/condition17.cfm (archived link [here](https://web.archive.org/web/20180924212805/https://www.fhwa.dot.gov/bridge/nbi/no10/condition17.cfm)), which lists 47,619 bridges as being in "poor" condition out of a total of 615,002 bridges nationwide.	

In [13]:
print(paste(round(47619/615002 * 100, 1), "percent of bridges nationwide were listed in 'poor' condition."))

[1] "7.7 percent of bridges nationwide were listed in 'poor' condition."


### Finding: Baltimore City owns all of the untolled bridges within its boundaries.

Filter the data to Baltimore City bridges and display the owners.

In [14]:
data.2017.clean %>% filter(county == 'Baltimore city') %>% select(owner) %>% table()

.
city or municipal highway agency             state toll authority 
                             241                              136 

The "city or municipal highway agency" (in Baltimore's case, the Baltimore City Department of Transportation) owns all of the untolled bridges inside Baltimore city (the tolled bridges are owned by the "state toll authority" which in Maryland's case is the Maryland Transportation Authority).

We can contrast this with other counties. For example, the bridges of Baltimore County are owned by a variety of agencies.

In [15]:
data.2017.clean %>% filter(county == 'Baltimore County') %>% select(owner) %>% table()

.
city or municipal highway agency            county highway agency 
                               8                              326 
            other state agencies    private (other than railroad) 
                               1                                3 
            state highway agency             state toll authority 
                             381                               59 

### Finding: 33 (13 percent) of city-owned bridges in Baltimore were listed in "poor" condition.

Filter the data to Baltimore City bridges owned by the city in poor condition, group by `bridge_condition` and calculate the total number and percentage of bridges in each category.

Note that the Wicomico Street bridge (`STRUCTURE_NUMBER_008 = '200000BC8032010'`) was dismantled last year, nearly a decade after railroad conglomerate CSX had [promised](http://articles.baltimoresun.com/2007-10-12/news/0710120088_1_avenue-bridge-replace-bridge-csx) to tear it down. We include it in the story because it was still listed in the FHA's 2017 dataset and had spent at least a decade in disrepair before being dismantled. For more details on the current status of some bridges, see the map of peristently "poor" bridges in the [story](http://www.baltimoresun.com/news/maryland/bs-md-bridge-collapse-maryland-20180815-story.html).

In [16]:
baltcity.conditions.2017 <- data.2017.clean %>% filter(county == 'Baltimore city' & 
                                                       owner == 'city or municipal highway agency') %>% 
                                            group_by(bridge_condition) %>% 
                                            summarise(n = n()) %>%
                                            mutate(perc = n/sum(n) * 100)

In [17]:
baltcity.conditions.2017

bridge_condition,n,perc
Fair,171,70.95436
Good,37,15.35270
Poor,33,13.69295


In [18]:
print(paste(round(baltcity.conditions.2017[baltcity.conditions.2017$bridge_condition == 'Poor',]$perc, 1), 
            "percent of city-owned bridges in Baltimore were listed in 'poor' condition."))

[1] "13.7 percent of city-owned bridges in Baltimore were listed in 'poor' condition."


### Finding: 20 city-owned bridges in Baltimore have been in "poor" condition for 10 years or more. 

Read in the 2007 data for comparison to 2017.

In [19]:
data.2007.clean <- read_feather('output/data_2007_clean.feather')

Create a dataframe, `baltcity.poor.2007`, with all of the city-owned bridges that were in "poor" condition in 2007 and another dataframe, `baltcity.poor.2017`, with all of the city-owned bridges that were in "poor" condition in 2017.

Note that the Wicomico Street bridge (`STRUCTURE_NUMBER_008 = '200000BC8032010'`) was dismantled last year, nearly a decade after railroad conglomerate CSX had [promised](http://articles.baltimoresun.com/2007-10-12/news/0710120088_1_avenue-bridge-replace-bridge-csx) to tear it down. We include it in the story because it was still listed in the FHA's 2017 dataset and had spent at least a decade in disrepair before being dismantled. For more details on the current status of some bridges, see the map of peristently "poor" bridges in the [story](http://www.baltimoresun.com/news/maryland/bs-md-bridge-collapse-maryland-20180815-story.html).

In [24]:
baltcity.poor.2007 <- data.2007.clean %>% filter(county == 'Baltimore city' & 
                                                 owner == 'city or municipal highway agency' & 
                                                 bridge_condition == 'Poor') 

baltcity.poor.2017 <- data.2017.clean %>% filter(county == 'Baltimore city' & 
                                                 owner == 'city or municipal highway agency' & 
                                                 bridge_condition == 'Poor') 

Merge the two dataframes and save into another dataframe called `baltcity.poor`, which will provide the number of bridges that were listed in "poor" condition in 2007 and were still in "poor" condition in 2017.

In [25]:
baltcity.poor <- merge(baltcity.poor.2007, baltcity.poor.2017, 
                      by = 'STRUCTURE_NUMBER_008', suffixes = c('_07', '_17'))

In [26]:
length(baltcity.poor$STRUCTURE_NUMBER_008)

[1] 20

In [27]:
print(paste(length(baltcity.poor$STRUCTURE_NUMBER_008), 
           "city-owned bridges in Baltimore have been in 'poor' condition for 10 years or more."))

[1] "20 city-owned bridges in Baltimore have been in 'poor' condition for 10 years or more."


### Finding: There were 36 bridges listed in “poor” condition in Baltimore County.

Filter the data to Baltimore County bridges to get the number of bridges that were listed in "poor" condition.

In [28]:
baltco.conditions.2017 <- data.2017.clean %>% filter(county == 'Baltimore County') %>% 
                                            group_by(bridge_condition) %>%
                                            summarise(n = n()) %>%
                                            mutate(perc = n/sum(n) * 100)

In [29]:
baltco.conditions.2017 <- data.2017.clean %>% filter(county == 'Baltimore County') %>% 
                                            group_by(bridge_condition) %>%
                                            summarise(n = n()) %>%
                                            mutate(perc = n/sum(n) * 100)

In [30]:
baltco.conditions.2017

bridge_condition,n,perc
Fair,527,67.737789
Good,215,27.634961
Poor,36,4.627249


In [32]:
print(paste("There were", baltco.conditions.2017[baltco.conditions.2017$bridge_condition == 'Poor',]$n,
            "bridges in 'poor' condition in Baltimore County."))

[1] "There were 36 bridges in 'poor' condition in Baltimore County."


### Finding: 11 bridges in Baltimore County have been in “poor” condition for 10 years or more.

Create a dataframe, `baltco.poor.2007`, with all of the bridges in Baltimore County that were in "poor" condition in 2007 and another dataframe, `baltco.poor.2017`, with all of the bridges in Baltimore County that were in "poor" condition in 2017.

In [33]:
baltco.poor.2007 <- data.2007.clean %>% filter(county == 'Baltimore County' & 
                                                 bridge_condition == 'Poor')

baltco.poor.2017 <- data.2017.clean %>% filter(county == 'Baltimore County' & 
                                                 bridge_condition == 'Poor')

Merge the two dataframes and save into another dataframe called `baltco.poor`, which will provide the number of bridges that were list in "poor" condition in 2007 and were still in "poor" condition in 2017.

In [34]:
baltco.poor <- merge(baltco.poor.2007, baltco.poor.2017, 
                      by = 'STRUCTURE_NUMBER_008', suffixes = c('_07', '_17'))

In [35]:
length(baltco.poor$STRUCTURE_NUMBER_008)

[1] 11

In [36]:
print(paste(length(baltco.poor$STRUCTURE_NUMBER_008), 
           "bridges in Baltimore County have been in 'poor' condition for 10 years or more."))

[1] "11 bridges in Baltimore County have been in 'poor' condition for 10 years or more."


### Finding: 21 bridges in Harford County were listed in “poor” condition.

Filter the data to Harford County bridges to get the number of bridges that are in "poor" condition.

In [37]:
harfordco.poor.2017 <- data.2017.clean %>% filter(county == 'Harford County' & 
                                                 bridge_condition == 'Poor') 

In [38]:
print(paste("There were", length(harfordco.poor.2017$STRUCTURE_NUMBER_008), 
      "bridges in Harford County listed in 'poor' condition."))

[1] "There were 21 bridges in Harford County listed in 'poor' condition."


### 11 bridges in Harford County have been in “poor” condition for 10 years or more.

Create a dataframe, `harfordco.poor.2007`, with all of the bridges in Harford County that were in "poor" condition in 2007.

In [39]:
harfordco.poor.2007 <- data.2007.clean %>% filter(county == 'Harford County' & 
                                                 bridge_condition == 'Poor') 

Merge this dataframe with `harfordco.poor.2017` and save into another dataframe called `harfordco.poor`, which will provide the number of bridges that were list in "poor" condition in 2007 and were still in "poor" condition in 2017.

In [40]:
harfordco.poor <- merge(harfordco.poor.2007, harfordco.poor.2017, 
                      by = 'STRUCTURE_NUMBER_008', suffixes = c('_07', '_17'))

In [41]:
print(paste(length(harfordco.poor$STRUCTURE_NUMBER_008), 
           "bridges in Harford County have been in poor condition for 10 years or more."))

[1] "11 bridges in Harford County have been in poor condition for 10 years or more."


### Finding: The State Highway Administration maintains roughly half of Maryland’s more than 5,000 bridges.

Filter the data to bridges owned or maintained by the State Highway Administration (either `owner` or `responsibility` listed as "state highway agency."

In [42]:
sha.owned <- data.2017.clean %>% filter(owner == 'state highway agency' |
                                        responsibility == 'state highway agency')

In [43]:
round(length(sha.owned$STRUCTURE_NUMBER_008)/length(data.2017.clean$STRUCTURE_NUMBER_008) * 100)

[1] 48

In [44]:
print(paste("The State Highway Administration maintains",
      round(length(sha.owned$STRUCTURE_NUMBER_008)/length(data.2017.clean$STRUCTURE_NUMBER_008) * 100),
      "percent (or roughly half) of Maryland's", length(data.2017.clean$STRUCTURE_NUMBER_008), "bridges."))

[1] "The State Highway Administration maintains 48 percent (or roughly half) of Maryland's 5335 bridges."


### Finding: The other half of the state’s bridges are owned by up to 15 different types of agencies, including county highway agencies, city or municipal highway agencies, state toll authorities, private owners and others.

Filter the data to owners or maintainers that are not the State Highway Administration and table the owners.

In [45]:
nonstate.owners <- data.2017.clean %>% filter(owner != 'state highway agency' & 
                           responsibility != 'state highway agency') %>% select(owner) %>% unique() %>% 
                    as.data.frame() %>% mutate(row_number = row_number())

nonstate.owners

owner,row_number
city or municipal highway agency,1
private (other than railroad),2
county highway agency,3
National Park Service,4
state toll authority,5
other state agencies,6
Navy/Marines,7
Bureau of Fish and Wildlife,8
Army,9
other local agencies,10


In [46]:
print(paste("Other than the State Highway Administration,", max(nonstate.owners$row_number), 
            "different agencies own Maryland's bridges."))

[1] "Other than the State Highway Administration, 15 different agencies own Maryland's bridges."


Below is the distribution of bridge ownership.

In [47]:
data.2017.clean %>% group_by(owner) %>% 
                    summarise(n = n()) %>% 
                    mutate(perc = n/sum(n) * 100) %>% arrange(desc(perc))

owner,n,perc
state highway agency,2545,47.70384255
county highway agency,1937,36.30740394
city or municipal highway agency,366,6.86035614
state toll authority,322,6.03561387
National Park Service,54,1.01218369
other state agencies,32,0.59981256
town or township highway agency,18,0.33739456
"local park, forest, or reservation agency",16,0.29990628
Army,11,0.20618557
Navy/Marines,10,0.18744142


### Finding: There is a bridge owned by NASA — a federal access road in Prince George’s County.

Filter the data to the bridge listed as owned by NASA.

In [48]:
data.2017.clean %>% filter(owner == 'NASA')

STRUCTURE_NUMBER_008,county,combinedfips,location,intersects,carries,route_type,service_level,responsibility,owner,⋯,avg_daily_traffic_future_yr,scour_critical,national_register,LAT_016,LONG_017,SCOUR_CRITICAL_113,COUNTY_CODE_003,structural_eval,waterway_eval,age
2170001P0000000,Prince George's County,24033,ROUTE 295 NORTH 3RD EXIT,BALTIMORE-WASHINGTON PKW,NASA ACCESS ROAD,Federal lands road,business,NASA,NASA,⋯,2030,not applicable,Bridge is not eligible for the National Register of Historic Places,39002050,076514500,N,033,5,NA,52


[Here](https://www.google.com/maps/place/39%C2%B000'20.5%22N+76%C2%B051'45.0%22W/@39.0060068,-76.8637175,172m/data=!3m1!1e3!4m5!3m4!1s0x0:0x0!8m2!3d39.00569!4d-76.8625) is the Google Satellite view of the bridge, which is located close to NASA's Goddard Space Flight Center.




Note: The maps featured in the online version the story were produced in CartoDB, after converting the `LAT_016` and `LONG_017` fields in the data and determining the validity of the coordinates provided. Not all bridges had valid geographical coordinates that could be mapped. For more information, see the mapping notebook [`03_mapping.ipynb`](https://github.com/baltimore-sun-data/bridge-data/blob/master/03_mapping.ipynb).